In [54]:
import numpy as np
import pandas as pd
from keras import optimizers
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras.src.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer, LabelEncoder

## Importation des données

In [55]:
data_train = pd.read_csv('Data/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_training-set.csv',
                         low_memory=False)
data_test = pd.read_csv('Data/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_testing-set.csv',
                        low_memory=False)
# x = dataset.iloc[:, 1:44].values
# y = dataset.iloc[:, 44].values

### Prétraitement des données
Cette fonction est conçue pour prétraiter les données en vue d'une utilisation avec des réseaux de neurones convolutionnels (CNN). Elle prend en entrée un tableau data et réalise les étapes suivantes :
- Normalisation des features : Les caractéristiques sélectionnées sont converties en float et normalisé pour avoir des valeurs comprises entre 0 et 1. Cette normalisation est essentielle pour s'assurer que toutes les caractéristiques ont le même poids lors de l'entraînement du modèle.

- Encodage des Étiquettes : Les étiquettes textuelles de l'ensemble de données sont d'abord transformées en étiquettes numériques à l'aide d'un LabelEncoder. Exemple : 'Normal' devient 0, 'Exploits' devient 1, etc. Ensuite, pour permettre une classification multiclasse, ces étiquettes numériques sont transformées en étiquettes binaires à l'aide d'un LabelBinarizer. Ce qui permet d'obtenir de nouvelles colonnes pour chaque étiquette, avec des valeurs binaires (0 ou 1) indiquant si l'échantillon appartient à cette classe ou non.

- Adaptation à l'entrée CNN: Les CNN traitent des images en 3D, avec une hauteur, une largeur et des canaux (comme les couleurs RGB). Nos données sont en 1D, donc nous les transformons pour qu'elles ressemblent à de petites "images" 3D. Chaque échantillon devient une "image" avec la longueur des caractéristiques comme hauteur, une largeur de 1 et un canal unique. Ainsi, nous pouvons utiliser un CNN sur des données qui ne sont pas vraiment des images.

La fonction renvoie finalement deux tableaux : x, qui contient les caractéristiques prétraitées et remodelées, et y, qui contient les étiquettes binarisées. La forme 3D de x est particulièrement adaptée pour être utilisée comme entrée dans un réseau de neurones convolutionnel.

In [56]:
def preprocess(data):
    # Initialisation des outils de prétraitement
    scaler = MinMaxScaler()  # Transformation des données en valeurs comprises entre 0 et 1
    label_encoder = LabelEncoder()  # Transformation des étiquettes textuelles en étiquettes numériques
    label_binarizer = LabelBinarizer()  # Transformation des étiquettes multiclasse en étiquettes binaires

    # Sélection des caractéristiques spécifiées pour l'ensemble de test
    x = data[:, [1, 6, 7, 8, 9, 10, 11, 12, 13, 27, 28, 32, 33, 34, 35, 36]]

    # Conversion des données en flottants et normalisation
    x = x.astype(float)
    scaler.fit(x)  # Calcul des paramètres de normalisation
    x = scaler.transform(x)  # Normalisation des données

    # Récupération des étiquettes de l'ensemble de test
    label = data[:, 43]  # 43 = attack_cat

    # Encodage des étiquettes textuelles en étiquettes numériques
    label_encoder.fit(label)
    y = label_encoder.transform(label)

    # Transformation des étiquettes numériques en étiquettes binaires
    label_binarizer.fit(y)
    y = label_binarizer.transform(y)

    # Remodelage des données pour les adapter à l'entrée 3D attendue par CNN
    x_final = []
    size = np.size(x, axis=1)
    for sample in x:
        reshaped_sample = sample.reshape([size, 1, 1])
        x_final.append(reshaped_sample)
    x = np.array(x_final)

    # La fonction renvoie les données prétraitées pour les caractéristiques et les étiquettes
    return x, y

In [57]:
x_train, y_train = preprocess(data_train.values)
x_test, y_test = preprocess(data_test.values)

shape = np.size(x_train, axis=1)

In [58]:
nb_classes = y_train.shape[1]

## Création du modèle

In [59]:
model = Sequential()  # Initialisation du modèle

### Bloc 1
## Deux couches de convolution avec une fonction d'activation ReLU
# 64 filtres de taille 3x1
# input_shape = (16 caractéristiques, 1 hauteur, 1 canal)
model.add(Conv2D(64, (3, 1), activation='relu', input_shape=(shape, 1, 1)))
model.add(Conv2D(64, (3, 1), activation='relu'))

## Couche de Pooling
# Réduit la complexité du modèle en réduisant la taille des données
# 2x1 filtre → réduit de 2 la hauteur et conserve la largeur
model.add(MaxPooling2D(pool_size=(2, 1)))

## Trois couches de convolution
# 128 filtres de taille 3x1
# padding="same" → assure que la sortie à la même taille que l'entrée
# ce qui permet d'éviter la perte d'information
model.add(Conv2D(128, (3, 1), activation='relu'))
model.add(Conv2D(128, (3, 1), activation='relu', padding="same"))
model.add(Conv2D(128, (3, 1), activation='relu', padding="same"))

## Couche de Pooling
model.add(MaxPooling2D(pool_size=(2, 1)))

## Trois couches de convolution
model.add(Conv2D(256, (3, 1), activation='relu', padding="same"))
model.add(Conv2D(256, (3, 1), activation='relu', padding="same"))
model.add(Conv2D(256, (3, 1), activation='relu', padding="same"))

## Couche de Pooling
model.add(MaxPooling2D(pool_size=(2, 1)))

# Aplatir les données en un vecteur 1D (actuellement 3D)
model.add(Flatten())

### Bloc 2
## Deux couches entièrement connectées
# 100 neurones
model.add(Dense(100, kernel_initializer='normal', activation='relu'))
# Dropout → évite le surapprentissage
# 50% des neurones sont ignorés aléatoirement à chaque itération, force le modèle à apprendre de nouvelles représentations
model.add(Dropout(0.5))
# name='output' → nomme la couche pour pouvoir y accéder plus tard
model.add(Dense(20, kernel_initializer='normal', activation='relu', name='output'))

# Couche de sortie
# 10 neurones → 10 classes
# softmax → fonction d'activation pour les problèmes de classification multiclasse
model.add(Dense(nb_classes, kernel_initializer='normal', activation='softmax'))


## Compilation du modèle

In [60]:
# Optimiseur Adam → algorithme de descente de gradient stochastique
# metrics=['categorical_accuracy'] → mesure la précision du modèle
opt = optimizers.Adam()
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['categorical_accuracy'])

# Permet d'arrêter l'entraînement si l'accuracy ne s'améliore plus (3 itérations)
stopper = EarlyStopping(monitor='val_binary_accuracy', patience=3, mode='auto')

# Entraînement du modèle
model.fit(x_train, y_train, epochs=1, batch_size=50, validation_data=(x_train, y_train), callbacks=[stopper])

1644/1647 [============================>.] - ETA: 0s - loss: 0.9199 - categorical_accuracy: 0.6657WARNING:tensorflow:Early stopping conditioned on metric `val_binary_accuracy` which is not available. Available metrics are: loss,categorical_accuracy,val_loss,val_categorical_accuracy


1647/1647 [==============================] - 17s 10ms/step - loss: 0.9195 - categorical_accuracy: 0.6659 - val_loss: 0.6701 - val_categorical_accuracy: 0.7660


## Sauvegarde du modèle

In [61]:
model.save('model.keras')

## Évaluation du modèle

In [64]:
score = model.evaluate(x_test, y_test, verbose=1)
print("Erreur sur les données de test :", score[0])
print("Précision sur les données de test :", score[1]*100, "%")

5480/5480 [==============================] - 9s 2ms/step - loss: 0.8476 - categorical_accuracy: 0.7035
Erreur sur les données de test : 0.8475897312164307
Précision sur les données de test : 70.35148739814758 %


## Création du modèle (Tentative 1)

In [ ]:
classifier = Sequential()

# Couche de convolution initiale
# 32 filtres de taille 3x3
# 'relu' est utilisé comme fonction d'activation pour ajouter de la non-linéarité
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))

# Couche de pooling pour réduire la dimensionnalité tout en conservant les caractéristiques
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Seconde couche de convolution
# Utilise également 32 filtres de taille 3x3
classifier.add(Conv2D(32, (3, 3), activation='relu'))

# Flattening: transforme le format 2D de la carte des caractéristiques en un vecteur 1D
classifier.add(Flatten())

# Première couche entièrement connectée (Dense)
# 128 neurones, fonction d'activation 'relu'
classifier.add(Dense(units=128, activation='relu'))

# Dropout pour éviter le surapprentissage. Désactive 50% des neurones de manière aléatoire pendant chaque itération d'entraînement.
classifier.add(Dropout(0.5))

# Couche de sortie
# Une seule unité pour une classification binaire avec fonction d'activation sigmoïde pour obtenir une probabilité
classifier.add(Dense(units=1, activation='sigmoid'))

# Compilation du modèle
# Utilisation de l'optimiseur 'adam' pour la descente de gradient
# Utilisation de la fonction de perte 'binary_crossentropy' pour la classification binaire
# Utilisation de la métrique 'accuracy' pour évaluer les performances du modèle
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entraînement du modèle
# Supposons que `train_data` et `train_labels` soient vos données et labels d'entraînement
# et que `val_data` et `val_labels` soient vos données et labels de validation.
# epochs représente le nombre de fois où l'ensemble des données d'entraînement est utilisé pour l'apprentissage
# batch_size représente le nombre d'échantillons qui seront propagés à travers le réseau à la fois.
classifier.fit(x, y, epochs=10, batch_size=32, validation_split=0.2)